In [2]:
import os 
class_names=os.listdir('train')
from tensorflow.keras.models import load_model
class model:
    def __init__(self):
        self.model = load_model('class2')
    def predict(self, image):
        pred_vec=self.model.predict(image)
        return class_names[np.argmax(pred_vec)]

In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [14]:
def modeling(image_directory='images', initial_epochs=1, fine_tune_epochs=1,
             base_learning_rate = 0.0001):
    train_dir = "train"
    validation_dir = "test"

    BATCH_SIZE = 32
    IMG_SIZE = (64, 64)

    train_dataset = image_dataset_from_directory(
        train_dir,label_mode='categorical', shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE
    )


    validation_dataset = image_dataset_from_directory(
        validation_dir,label_mode='categorical', shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE
    )


    class_names = train_dataset.class_names

    val_batches = tf.data.experimental.cardinality(validation_dataset)
    test_dataset = validation_dataset.take(val_batches // 5)
    validation_dataset = validation_dataset.skip(val_batches // 5)


    data_augmentation = tf.keras.Sequential(
        [
             tf.keras.layers.experimental.preprocessing.RandomFlip("vertical"),
             tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
        ]
    )

    preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

    rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1.0 / 127.5, offset=-1)

    IMG_SHAPE = IMG_SIZE + (3,)
    base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

    image_batch, label_batch = next(iter(train_dataset))
    feature_batch = base_model(image_batch)
    print(feature_batch.shape)

    base_model.trainable = False

    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    feature_batch_average = global_average_layer(feature_batch)

    prediction_layer = tf.keras.layers.Dense(len(class_names))
    prediction_batch = prediction_layer(feature_batch_average)

    inputs = tf.keras.Input(shape=(64, 64, 3))
    x = data_augmentation(inputs)
    x = preprocess_input(x)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )
    
    history = model.fit(train_dataset, epochs=initial_epochs, validation_data=validation_dataset)

    base_model.trainable = True

    fine_tune_at = 100

    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate / 10),
        metrics=["accuracy"],
    )
    total_epochs = initial_epochs + fine_tune_epochs
    history_fine = model.fit(
        train_dataset, epochs=total_epochs, initial_epoch=history.epoch[-1], validation_data=validation_dataset
    )
    
    model.save('class2')

In [15]:
modeling()

Found 4800 files belonging to 4 classes.
Found 1600 files belonging to 4 classes.
(32, 2, 2, 1280)
150/150 [==============================] - 67s 353ms/step - loss: 1.9933 - accuracy: 0.3007 - val_loss: 1.0972 - val_accuracy: 0.5758
Epoch 1/2
150/150 [==============================] - 91s 449ms/step - loss: 1.0153 - accuracy: 0.6010 - val_loss: 0.4545 - val_accuracy: 0.8117
Epoch 2/2
150/150 [==============================] - 67s 441ms/step - loss: 0.5992 - accuracy: 0.7533 - val_loss: 0.3681 - val_accuracy: 0.8445
INFO:tensorflow:Assets written to: class2\assets


In [5]:
model=tf.keras.models.load_model('class2')

In [11]:
from tensorflow.keras.preprocessing import image
img=image.load_img('train/bird/1001_128.jpg', target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
model.predict(x)

array([[-5.1758766,  4.0067916, -3.075168 , -8.931844 ]], dtype=float32)

In [12]:
x.shape

(1, 64, 64, 3)

In [13]:
os.listdir('train')

['berry', 'bird', 'dog', 'flower']

In [1]:
from train import modeling


In [3]:
modeling()

Found 4800 files belonging to 4 classes.
Found 1600 files belonging to 4 classes.
(32, 2, 2, 1280)
150/150 [==============================] - 40s 236ms/step - loss: 2.2094 - accuracy: 0.2876 - val_loss: 1.2143 - val_accuracy: 0.5180
Epoch 1/2
150/150 [==============================] - 44s 250ms/step - loss: 1.0902 - accuracy: 0.5742 - val_loss: 0.4315 - val_accuracy: 0.8328
Epoch 2/2
150/150 [==============================] - 37s 242ms/step - loss: 0.6020 - accuracy: 0.7516 - val_loss: 0.3479 - val_accuracy: 0.8758
INFO:tensorflow:Assets written to: class2\assets
